In [ ]:
from BIDS import BIDS_Global_info, BIDS_Family

In [ ]:
#Lets import the class that represent a whole data set.
from BIDS import BIDS_Global_info, BIDS_Family
# You can parse multiple datasets and select what parent folder are read (e.g. rawdata, derivatives)
bids_global_object = BIDS_Global_info(['/home/daniel/Documents/Uni/practical-sose23/castellvi/3D-Castellvi-Prediction/data/dataset-verse-combined'],['rawdata',"derivatives"], additional_key = ["ct", "subreg", "cortex"], verbose=True)
#The Parser will inform you about every non standard files. To add additional key add them to additional_key list, so you don't get the msg that this is not a valid key

from BIDS.snapshot2D import create_snapshot,Snapshot_Frame,Visualization_Type,Image_Modes
from pathlib import Path
from IPython.display import Image